# 1. Spark Basics

solutions: https://gist.github.com/yarax/b73fada9315a195c818367bd234649e5        

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("AWSpark").getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 1.1. Read data from from S3 s3a://aws-stocks-dataset/AAL.csv as dataframe

In [2]:
df = spark.read.option("header","true").csv("s3a://aws-stocks-dataset/AAL.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------+------------------+------------------+------------------+
|      Date|               Low|              Open| Volume|              High|             Close|    Adjusted Close|
+----------+------------------+------------------+-------+------------------+------------------+------------------+
|27-09-2005|19.100000381469727|21.049999237060547| 961200|21.399999618530273|19.299999237060547|18.194910049438477|
|28-09-2005|19.200000762939453|19.299999237060547|5747900|20.530000686645508|              20.5|19.326204299926758|
|29-09-2005|20.100000381469727|20.399999618530273|1078200|20.579999923706055|20.209999084472656| 19.05280303955078|
|30-09-2005| 20.18000030517578|20.260000228881836|3123300|21.049999237060547|21.010000228881836|19.806997299194336|
|03-10-2005|20.899999618530273|20.899999618530273|1057900|             21.75|              21.5|20.268939971923828|
|04-10-2005|21.440000534057617|21.440000534057617|1768800|              

In [4]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Date: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adjusted Close: string (nullable = true)

### 1.2. Convert the date field from string to date (create a new column date_dt)

In [5]:
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Date', 'Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close']

In [6]:
from pyspark.sql.functions import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# create a new column 
df = df.withColumn("Date_dt", df["Date"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Date', 'Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close', 'Date_dt']

In [9]:
df = df.withColumn("Date_dt", to_date(col('Date'), "dd-MM-yyyy"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+
|      Date|               Low|              Open| Volume|              High|             Close|    Adjusted Close|   Date_dt|
+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+
|27-09-2005|19.100000381469727|21.049999237060547| 961200|21.399999618530273|19.299999237060547|18.194910049438477|2005-09-27|
|28-09-2005|19.200000762939453|19.299999237060547|5747900|20.530000686645508|              20.5|19.326204299926758|2005-09-28|
|29-09-2005|20.100000381469727|20.399999618530273|1078200|20.579999923706055|20.209999084472656| 19.05280303955078|2005-09-29|
|30-09-2005| 20.18000030517578|20.260000228881836|3123300|21.049999237060547|21.010000228881836|19.806997299194336|2005-09-30|
|03-10-2005|20.899999618530273|20.899999618530273|1057900|             21.75|              21.5|20.268939971923

### 1.3. Convert the "high" column to cents (create a newcolumn high_cent)

In [11]:
# create a new column by copying an existing column
df = df.withColumn("High_cent", df.High*100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+------------------+
|      Date|               Low|              Open| Volume|              High|             Close|    Adjusted Close|   Date_dt|         High_cent|
+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+------------------+
|27-09-2005|19.100000381469727|21.049999237060547| 961200|21.399999618530273|19.299999237060547|18.194910049438477|2005-09-27|2139.9999618530273|
|28-09-2005|19.200000762939453|19.299999237060547|5747900|20.530000686645508|              20.5|19.326204299926758|2005-09-28| 2053.000068664551|
|29-09-2005|20.100000381469727|20.399999618530273|1078200|20.579999923706055|20.209999084472656| 19.05280303955078|2005-09-29|2057.9999923706055|
|30-09-2005| 20.18000030517578|20.260000228881836|3123300|21.049999237060547|21.010000228881836|19.806997299194336|2005-09-3

### 1.4. Print the top 10 stocks

In [13]:
#df.sort("Open").show(10) # can only sort in asc order 

df.orderBy(df.Open.desc()).show(10) # --> nice try, but the numbers are evaluated by the first number.. not by the actual value

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+-----------------+--------+------------------+------------------+------------------+----------+------------------+
|      Date|              Low|             Open|  Volume|              High|             Close|    Adjusted Close|   Date_dt|         High_cent|
+----------+-----------------+-----------------+--------+------------------+------------------+------------------+----------+------------------+
|20-01-2011|9.670000076293945|9.979999542236328| 7261600|10.300000190734863|             10.25| 9.663102149963379|2011-01-20|1030.0000190734863|
|01-02-2011| 9.90999984741211|9.970000267028809| 7210700|10.130000114440918|  9.90999984741211| 9.342570304870605|2011-02-01|1013.0000114440918|
|29-12-2010|9.850000381469727|9.949999809265137| 2678200| 10.15999984741211|10.140000343322754| 9.559399604797363|2010-12-29|1015.9999847412109|
|09-02-2011|9.760000228881836|9.949999809265137| 5590900| 9.960000038146973| 9.880000114440918| 9.314288139343262|2011-02-09| 996.

In [14]:
# check data type
df.select('Open').printSchema()

# its string --> convert to float

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Open: string (nullable = true)

In [15]:
# https://sparkbyexamples.com/pyspark/pyspark-convert-string-type-to-double-type-float-type/ 
from pyspark.sql.types import DoubleType 
df = df.withColumn("Open", df.Open.cast(DoubleType()))
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Date: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adjusted Close: string (nullable = true)
 |-- Date_dt: date (nullable = true)
 |-- High_cent: double (nullable = true)

In [16]:
df.orderBy(df.Open.desc()).show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+-----------------+
|      Date|               Low|              Open| Volume|              High|             Close|    Adjusted Close|   Date_dt|        High_cent|
+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+-----------------+
|27-11-2006|58.779998779296875| 62.70000076293945|1693400| 62.79999923706055| 59.15999984741211|  55.7725715637207|2006-11-27|6279.999923706055|
|24-11-2006|              62.0| 62.04999923706055| 664600| 63.27000045776367| 62.95000076293945| 59.34557342529297|2006-11-24|6327.000045776367|
|22-11-2006|60.810001373291016|61.400001525878906|2583800|63.099998474121094| 62.95000076293945| 59.34557342529297|2006-11-22|6309.999847412109|
|13-12-2006|58.630001068115234| 61.33000183105469|3147200| 62.45000076293945|             59.25| 55.85742950439453|2006-12-13|6245

### 1.5. Save the dataframe locally as parquet

In [17]:
df = df.withColumnRenamed("Adjusted Close", "Adjusted_close")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# https://www.ibm.com/support/pages/how-do-you-write-spark-dataframe-parquet-file-format-cloud-object-storage
df.write.parquet("/media/ubi20/SanDisk/code/data_engineer/03_module_etl2_spark/spark_basics.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df.write.mode("overwrite").parquet("stocks.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 1.6. Read the saved data as rdd and multiply every "High" values by 2

In [20]:
df_parquet = spark.read.parquet("stocks.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df_parquet.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+------------------+
|      Date|               Low|              Open| Volume|              High|             Close|    Adjusted_close|   Date_dt|         High_cent|
+----------+------------------+------------------+-------+------------------+------------------+------------------+----------+------------------+
|27-09-2005|19.100000381469727|21.049999237060547| 961200|21.399999618530273|19.299999237060547|18.194910049438477|2005-09-27|2139.9999618530273|
|28-09-2005|19.200000762939453|19.299999237060547|5747900|20.530000686645508|              20.5|19.326204299926758|2005-09-28| 2053.000068664551|
|29-09-2005|20.100000381469727|20.399999618530273|1078200|20.579999923706055|20.209999084472656| 19.05280303955078|2005-09-29|2057.9999923706055|
|30-09-2005| 20.18000030517578|20.260000228881836|3123300|21.049999237060547|21.010000228881836|19.806997299194336|2005-09-3

In [22]:
rdd = spark.sparkContext.textFile("s3a://aws-stocks-dataset/AAL.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
sc.textFile("s3a://aws-stocks-dataset/AAL.csv").map(lambda line: line.split(","))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[45] at RDD at PythonRDD.scala:53

In [24]:
rdd.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Date,Low,Open,Volume,High,Close,Adjusted Close', '27-09-2005,19.100000381469727,21.049999237060547,961200,21.399999618530273,19.299999237060547,18.194910049438477', '28-09-2005,19.200000762939453,19.299999237060547,5747900,20.530000686645508,20.5,19.326204299926758', '29-09-2005,20.100000381469727,20.399999618530273,1078200,20.579999923706055,20.209999084472656,19.05280303955078', '30-09-2005,20.18000030517578,20.260000228881836,3123300,21.049999237060547,21.010000228881836,19.806997299194336', '03-10-2005,20.899999618530273,20.899999618530273,1057900,21.75,21.5,20.268939971923828', '04-10-2005,21.440000534057617,21.440000534057617,1768800,22.5,22.15999984741211,20.891155242919922', '05-10-2005,21.75,22.100000381469727,904300,22.309999465942383,22.200000762939453,20.928861618041992', '06-10-2005,22.399999618530273,22.600000381469727,1614200,23.0,22.579999923706055,21.2871036529541', '07-10-2005,21.799999237060547,22.25,753400,22.600000381469727,22.149999618530273,20.881725311279297']

### 1.7. Read the data and try to query and filter it with Spark SQL

In [27]:
# rdd.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df = spark.read.option("header","true").csv("s3a://aws-stocks-dataset/").limit(100000)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+------------------+------+-------------------+-------------------+-------------------+
|      Date|                Low|              Open|Volume|               High|              Close|     Adjusted Close|
+----------+-------------------+------------------+------+-------------------+-------------------+-------------------+
|21-02-1973|0.39506199955940247|               0.0| 15188|0.39506199955940247|0.39506199955940247|0.39506199955940247|
|22-02-1973| 0.3703700006008148|               0.0|  9113| 0.3703700006008148| 0.3703700006008148| 0.3703700006008148|
|23-02-1973|0.34567898511886597|               0.0|  3038|0.34567898511886597|0.34567898511886597|0.34567898511886597|
|26-02-1973|0.34567898511886597|               0.0|  1519|0.34567898511886597|0.34567898511886597|0.34567898511886597|
|27-02-1973|0.34567898511886597|               0.0| 29869|0.34567898511886597|0.34567898511886597|0.34567898511886597|
|28-02-1973|0.34567898511886597|               0

In [27]:
df = df.withColumn("Date_dt", to_date(col('Date'), "YYYY-MM-DD"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+----+------+-------------------+-------------------+-------------------+-------+
|      Date|                Low|Open|Volume|               High|              Close|     Adjusted Close|Date_dt|
+----------+-------------------+----+------+-------------------+-------------------+-------------------+-------+
|21-02-1973|0.39506199955940247| 0.0| 15188|0.39506199955940247|0.39506199955940247|0.39506199955940247|   null|
|22-02-1973| 0.3703700006008148| 0.0|  9113| 0.3703700006008148| 0.3703700006008148| 0.3703700006008148|   null|
|23-02-1973|0.34567898511886597| 0.0|  3038|0.34567898511886597|0.34567898511886597|0.34567898511886597|   null|
|26-02-1973|0.34567898511886597| 0.0|  1519|0.34567898511886597|0.34567898511886597|0.34567898511886597|   null|
|27-02-1973|0.34567898511886597| 0.0| 29869|0.34567898511886597|0.34567898511886597|0.34567898511886597|   null|
+----------+-------------------+----+------+-------------------+-------------------+------------